In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LogisticRegression

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_excel("/kaggle/input/online-retail/Online Retail.xlsx")

In [3]:
# In this lets see the first five row's
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# In this we will see the Total number of rows and columns
df.shape

(541909, 8)

In [5]:
# In this lets see the total columns name
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [6]:
# In this we check the dtype of the data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
# Lets us check the null values
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
# Dropping rows with null values
df = df.dropna()

In [9]:
# Now we have drop the null values
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
# Filter the data for the specific Customer ID and Date
customer_id = 17854
date = 2010-12-1
filtered_data = df[(df['CustomerID'] == customer_id) & (df['InvoiceDate'].dt.date == pd.to_datetime(date).date())]

In [11]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


In [12]:
customer_id = 17850.0
date = '2010-12-01 08:26:00'

filtered_data = df[(df['CustomerID'] == customer_id) & (df['InvoiceDate'] == pd.to_datetime(date))]

if filtered_data.empty:
    print("No data found for the specified Customer ID and Date.")
else:
    # Step 2: Training a Model
    X = filtered_data[['InvoiceNo', 'UnitPrice']]
    y = filtered_data['Quantity']

    # One-hot encode the 'StockCode' column
    #ct = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')
    #X_encoded = ct.fit_transform(X)
    
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0, 1])], remainder='passthrough')
    X_encoded = ct.fit_transform(X)

    model = LinearRegression()
    model.fit(X_encoded, y)

    # Step 3: Predicting Quantity
    new_data = pd.DataFrame({'InvoiceNo': ['NEW_INVOICE'], 'StockCode': ['NEW_STOCK'], 'UnitPrice': [0.0]})

    if not new_data.empty:
        new_data_encoded = ct.transform(new_data)

        prediction = model.predict(new_data_encoded)

        # Step 4: Print the Prediction
        print(f"Predicted quantity for Customer ID {customer_id} on {date}: {prediction[0]}")
    else:
        print("New data is empty. Unable to make a prediction.")

Predicted quantity for Customer ID 17850.0 on 2010-12-01 08:26:00: 5.6000000000000005


## Lets check the accuracy of the model

In [13]:
from sklearn.metrics import mean_squared_error

# Step 2: Training a Model
X = filtered_data[['InvoiceNo', 'UnitPrice']]
y = filtered_data['Quantity']

# One-hot encode the 'StockCode' column
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0, 1])], remainder='passthrough')
X_encoded = ct.fit_transform(X)

model = LinearRegression()
model.fit(X_encoded, y)

# Step 3: Predicting Quantity
new_data = pd.DataFrame({'InvoiceNo': ['NEW_INVOICE'], 'StockCode': ['NEW_STOCK'], 'UnitPrice': [0.0]})

if not new_data.empty:
    new_data_encoded = ct.transform(new_data)

    prediction = model.predict(new_data_encoded)

    # Step 4: Print the Prediction
    print(f"Predicted quantity for Customer ID {customer_id} on {date}: {prediction[0]}")
    
    # Step 5: Calculate Mean Squared Error (MSE)
    y_pred = model.predict(X_encoded)
    mse = mean_squared_error(y, y_pred)
    print(f"Mean Squared Error: {mse}")
else:
    print("New data is empty. Unable to make a prediction.")

Predicted quantity for Customer ID 17850.0 on 2010-12-01 08:26:00: 5.6000000000000005
Mean Squared Error: 3.6625684885261264e-31



## The above code performs a linear regression model to predict the quantity of items for a specific customer on a given date. Here's a      step-by-step description of the code.

1.The code reads an Excel file containing retail data using pandas.

2.It filters the data based on a specific customer ID and date to obtain a subset of records.

3.If the filtered data is not empty, the code proceeds to train a linear regression model to predict the quantity of items.

4.The features used for training are 'InvoiceNo' and 'UnitPrice', and the target variable is 'Quantity'.

5.The 'InvoiceNo' and 'UnitPrice' columns are encoded using one-hot encoding, while the remaining columns are left unchanged.

6.The linear regression model is fitted on the encoded features and target variable.

7.The code then creates new data with a hypothetical 'InvoiceNo', 'StockCode', and 'UnitPrice'.

8.If the new data is not empty, it is encoded using the same column transformer.

9.The trained model is used to predict the quantity of items for the new data.

10.Finally, the predicted quantity is printed as the output.

## Additionally, the code calculates the mean squared error (MSE) to evaluate the model's accuracy. A very low MSE value suggests that      the model's predictions are very close to the actual values.